# With Edward

In [4]:
import tensorflow_probability as tfp
from tensorflow_probability import edward2 as ed
import numpy as np

In [5]:
x_train = np.linspace(-3, 3, num=50)
y_train = np.cos(x_train) + np.random.normal(0, 0.1, size=50)
x_train = x_train.astype(np.float32).reshape((50, 1))
y_train = y_train.astype(np.float32).reshape((50, 1))

In [7]:
W_0 = ed.Normal(loc=tf.zeros([1, 2]), scale=tf.ones([1, 2]))
W_1 = ed.Normal(loc=tf.zeros([2, 1]), scale=tf.ones([2, 1]))
b_0 = ed.Normal(loc=tf.zeros(2), scale=tf.ones(2))
b_1 = ed.Normal(loc=tf.zeros(1), scale=tf.ones(1))

x = x_train
y = ed.Normal(loc=tf.matmul(tf.tanh(tf.matmul(x, W_0) + b_0), W_1) + b_1,
           scale=0.1)

In [8]:
qW_0 = ed.Normal(loc=tf.get_variable("qW_0/loc", [1, 2]),
              scale=tf.nn.softplus(tf.get_variable("qW_0/scale", [1, 2])))
qW_1 = ed.Normal(loc=tf.get_variable("qW_1/loc", [2, 1]),
              scale=tf.nn.softplus(tf.get_variable("qW_1/scale", [2, 1])))
qb_0 = ed.Normal(loc=tf.get_variable("qb_0/loc", [2]),
              scale=tf.nn.softplus(tf.get_variable("qb_0/scale", [2])))
qb_1 = ed.Normal(loc=tf.get_variable("qb_1/loc", [1]),
              scale=tf.nn.softplus(tf.get_variable("qb_1/scale", [1])))

In [12]:
# ed1
# inference = ed.KLqp({W_0: qW_0, b_0: qb_0,
#                     W_1: qW_1, b_1: qb_1}, data={y: y_train})
# inference.run(n_iter=1000)

# ed2
with ed.tape() as model_tape:
    with ed.interception(make_value_setter(W_0=qW_0, b_0=qb_0, W_1=qW_1, b_1=qb_1, z1=qz1, z0=qz0)):
        posterior_predictive = deep_exponential_family(data_size, feature_size, units, shape)

log_likelihood = posterior_predictive.distribution.log_prob(y_train)

# Compute analytic KL-divergence between variational and prior distributions.
kl = 0.
for rv_name, variational_rv in [("z0", qz0), ("z1", qz1), ("z2", qz2),
                                ("w0", qw0), ("w1", qw1), ("w2", qw2)]:
  kl += tf.reduce_sum(variational_rv.distribution.kl_divergence(
      model_tape[rv_name].distribution))

elbo = tf.reduce_mean(log_likelihood - kl)
tf.summary.scalar("elbo", elbo)
optimizer = tf.train.AdamOptimizer(1e-3)
train_op = optimizer.minimize(-elbo)

IndentationError: unexpected indent (<ipython-input-12-e3302c05ad7a>, line 3)